In [ ]:
!pip install datasets peft > null

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%pip install --quiet transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2

import math
import torch
import gc
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)

from pprint import pprint
from peft import (
    get_peft_config,
    get_peft_model,
    PeftModel,
    PromptTuningInit,
    PromptTuningConfig,
    TaskType,
    PeftType
)
import torch.nn as nn
import torch.nn.functional as F
import transformers
from tqdm.auto import tqdm, trange

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 35.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53

In [ ]:
pip install accelerate

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
# Инициализация токенизатора и модели
#MODEL_NAME = 'facebook/xglm-564M' #работает, но не всегда генерирует адекватные ответы
#MODEL_NAME = 'IlyaGusev/saiga_mistral_7b_lora' # не поместилась в колаб
MODEL_NAME = 'IlyaGusev/saiga2_7b_lora'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = transformers.AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)

for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module  # pre-trained (frozen) linear layer
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))

    def forward(self, input):
        # Apply self.module and LoRA adapter, return the sum (self.module outputs + adapter outputs)
        module_output = self.module(input)
        adapter_output = torch.matmul(input, self.adapter_A)
        adapter_output = torch.matmul(adapter_output, self.adapter_B)
        return module_output + adapter_output

In [ ]:
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LoRALayer(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "please check your forward pass"

test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)
dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "bad grad w.r.t. A"
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "bad grad w.r.t. B"
# note: bad grad means that your code is different from LoRA paper OR that your code is not autograd-friendly (e.g. no_grad)
del dummy_loss, test_linear, test_adapter
print("All tests passed!")

All tests passed!


Архитектура до лоры:

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): Linear4bit(
            in_features=4096, out_features=4096, bias=False
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(


Применим LoRA к Q/K/V линейным слоям в Llama attention. Вы также можете изменить другие слои

In [ ]:
lora_rank = 8

for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # for Llama-7B

Архитектура с лорой:

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): LoRALayer(
            (module): Linear4bit(
              in_features=4096, out_features=4096, bias=False
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.05, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=4096, out_features=16, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=16, out_features=4096, bias=False)
              )
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
            )
          )
          (k_proj): LoRALayer(
            (module): Linear4bit(
              in_features=4096, out_features=4096, bias=False
              (lora_dropout): ModuleDi

In [ ]:
batch = tokenizer("This model wants to share its greatest secret:", return_tensors='pt', return_token_type_ids=False)
#batch.to(device)
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast(dtype=torch.float32):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None
        assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)
print("Grad check successful, well done!")

Grad check successful, well done!


Подготовим данные

In [ ]:
dataset = load_dataset("dkagramanyan/horoscopes_ru")

tokenizer.pad_token = tokenizer.eos_token

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/66501 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6976 [00:00<?, ? examples/s]

In [ ]:
def tokenize(element, prompt="Гороскоп на завтра: "):
    if isinstance(element["text"], list):
        text = [prompt + s for s in element["text"]]
    else:
        text = prompt + element["text"]

    outputs = tokenizer(
        text,
        truncation=True,
        max_length=128,
        padding="max_length",
        return_tensors='pt',
        add_special_tokens=True
    )
    return outputs

In [ ]:
# Токенизация датасета
train_ds = dataset["train"].map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)

test_ds = dataset["test"].map(
    tokenize, batched=True, remove_columns=dataset["test"].column_names
)

Map:   0%|          | 0/66501 [00:00<?, ? examples/s]

Map:   0%|          | 0/6976 [00:00<?, ? examples/s]

In [ ]:
def predict(text="", model=model, **kwargs):
    model.eval()
    inputs = tokenizer(
        "Гороскоп на завтра: " + text,
        return_tensors='pt',
        add_special_tokens=True
    )

    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            eos_token_id=50257,
            **kwargs
        )
        return tokenizer.batch_decode(
            outputs.detach().cpu().numpy(),
            skip_special_tokens=True
            )

Предсказания модели до тренировки:

In [ ]:
result = predict(
    "Хорошее начало дня",
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
    )

result

['Гороскоп на завтра: Хорошее начало дня! bot\nПриветствую вас, дорогой посетитель! Я - программа искусственного интеллекта, которую вы используете для получения информации о будущих событиях.']

In [ ]:
result = predict(
    "Днем имеет смысл заняться",
    no_repeat_ngram_size=3,
    repetition_penalty=3.0,
    temperature=3
    )

result

['Гороскоп на завтра: Днем имеет смысл заняться делами, которые приносят пользу. Unterscheidung der Fälle von A bis Z ist nicht immer einfach und oftmals sehr kompliziert!\nГородской горизонтальный план города - это пла']

In [ ]:
result = predict(
    "Близнецам",
    no_repeat_ngram_size=1,
    num_beams=4,
    top_p=0.9,
    top_k=20,
    repetition_penalty=1.0
    )

result

['Гороскоп на завтра: Близнецам предстоит счастье и благополучия в любви, а также успешная карьера. Einzelnen werden Glück und Wohlstand in der Liebe sowie ein erfolgreiches Berufsleben beschert sein']

In [ ]:
result = predict(
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
    )

result

['Гороскоп на завтра: 2019 год\n Hinweis: Если вы хотите получить более точную информацию, пожалуйста, обратитесь к профессиональному психологу или астрологу.']

In [ ]:
result = predict(
    "Завтра вас ждет",
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
    )

result

['Гороскоп на завтра: Завтра вас ждет новый день, в котором вы сможете реализовать свои мечты и достигнуть своих целей! bot\nЗавтра будет дождливым днем. Не стоит']

Теперь обучим модель

In [ ]:
# checking if the model can learn. Change max_steps for proper training
import datasets
import transformers

model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=train_ds,
    eval_dataset=test_ds,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, gradient_accumulation_steps=1,
        # note: if you want larger batch size, increase gradient_accumulation_steps
        warmup_steps=250, max_steps=300, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='./outputs', overwrite_output_dir=True, report_to=None, save_total_limit=3, save_steps=500,),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

Step,Training Loss
1,2.074600
2,1.775400
3,1.863300
4,2.042000
5,1.729000
6,1.748400
7,1.914400
8,2.110200
9,1.796400
10,1.825400


TrainOutput(global_step=300, training_loss=1.6931627635161082, metrics={'train_runtime': 540.2172, 'train_samples_per_second': 1.111, 'train_steps_per_second': 0.555, 'total_flos': 3055293982310400.0, 'train_loss': 1.6931627635161082, 'epoch': 0.01})

Пример генерации

Оценка

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

KeyboardInterrupt: 

In [ ]:
def predict(text="", model=trainer.model, **kwargs):
    model.eval()
    inputs = tokenizer(
        "Гороскоп на завтра: " + text,
        return_tensors='pt',
        add_special_tokens=True
    )

    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            eos_token_id=50257,
            **kwargs
        )
        return tokenizer.batch_decode(
            outputs.detach().cpu().numpy(),
            skip_special_tokens=True
            )

In [ ]:
result = predict(
    "Хорошее начало дня",
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
    )

result

['Гороскоп на завтра: Хорошее начало дня не гарантирует успеха в дальнейшем развитии дел. Не исключено, что вы столкнетесь с ненужными ограничениями или препятстви']

In [ ]:
result = predict(
    "Днем имеет смысл заняться",
    no_repeat_ngram_size=3,
    repetition_penalty=3.0,
    temperature=3
    )

result

['Гороскоп на завтра: Днем имеет смысл заняться самосовершенствованием, улучшению профессиональных и личностно-нравственных качеств. В этот день вы можете получить новые знания или опыт в области вашего дела;']

In [ ]:
result = predict(
    "Близнецам",
    no_repeat_ngram_size=1,
    num_beams=4,
    top_p=0.9,
    top_k=20,
    repetition_penalty=1.0
    )

result

['Гороскоп на завтра: Близнецам сегодня не стоит упускать шанс, особенно в профессиональной сфере. Не исключены успешные поездки за границу или приобретение новых']

In [ ]:
result = predict(
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
    )

result

['Гороскоп на завтра:  Скорпионы могут столкнуться с трудностями, связанными с реализацией своих планов. Не исключено, что вам придется пересмотреть стратеги']

In [ ]:
result = predict(
    "Завтра вас ждет",
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
    )

result

['Гороскоп на завтра: Завтра вас ждет счастливый день. Не исключено, что вы почувствуете себя особенно энергичным и творчески подвижным в течение всего дня. Возможно,']

### Peft config Lora

In [ ]:
from peft import LoraConfig

model_id = 'IlyaGusev/saiga2_7b_lora'
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
model = get_peft_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(
                in_features=4096, out_features=4096, bias=Fals

### Подготовка данных

In [ ]:
dataset = load_dataset("dkagramanyan/horoscopes_ru")

tokenizer.pad_token = tokenizer.eos_token

def tokenize(element, prompt="Гороскоп на завтра: "):
    if isinstance(element["text"], list):
        text = [prompt + s for s in element["text"]]
    else:
        text = prompt + element["text"]

    outputs = tokenizer(
        text,
        truncation=True,
        max_length=128,
        padding="max_length",
        return_tensors='pt',
        add_special_tokens=True
    )
    return outputs

# Токенизация датасета
train_ds = dataset["train"].map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)

test_ds = dataset["test"].map(
    tokenize, batched=True, remove_columns=dataset["test"].column_names
)

In [ ]:
# checking if the model can learn. Change max_steps for proper training
import datasets
import transformers

model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=train_ds,
    eval_dataset=test_ds,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, gradient_accumulation_steps=1,
        # note: if you want larger batch size, increase gradient_accumulation_steps
        warmup_steps=250, max_steps=300, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='./outputs', overwrite_output_dir=True, report_to=None, save_total_limit=3, save_steps=500,),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# if you see cache warnings, set `model.config.use_cache = False` to silence them. Please re-enable for inference!

trainer.train()

# NOTE: это только пример! Вам не нужно ждать окончания!

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.661800
2,2.285200
3,2.377600
4,2.424900
5,2.239000
6,2.181800
7,2.454900
8,2.589600
9,2.323700
10,2.280600


TrainOutput(global_step=300, training_loss=1.7988063462575277, metrics={'train_runtime': 327.3002, 'train_samples_per_second': 1.833, 'train_steps_per_second': 0.917, 'total_flos': 3052394879385600.0, 'train_loss': 1.7988063462575277, 'epoch': 0.01})

In [ ]:
def predict(text="", model=trainer.model, **kwargs):
    model.eval()
    inputs = tokenizer(
        "Гороскоп на завтра: " + text,
        return_tensors='pt',
        add_special_tokens=True
    )

    with torch.no_grad():
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=50,
            eos_token_id=50257,
            **kwargs
        )
        return tokenizer.batch_decode(
            outputs.detach().cpu().numpy(),
            skip_special_tokens=True
            )

In [ ]:
result = predict(
    "Хорошее начало дня",
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=50,
    repetition_penalty=5.0
    )

result

['Гороскоп на завтра: Хорошее начало дня не гарантирует успеха в дальнейшем развитии событий. Не исключено, что вы столкнетесь с ненужными ограничениями или препят']

In [ ]:
result = predict(
    "Близнецам",
    no_repeat_ngram_size=1,
    num_beams=4,
    top_p=0.9,
    top_k=20,
    repetition_penalty=1.0
    )

result

['Гороскоп на завтра: Близнецам сегодня не стоит забывать о своем здоровье. Не исключено, что у вас возникнут проблемы с пищеварительным трактом (возможна болез']

In [ ]:
result = predict(
    no_repeat_ngram_size=3,
    num_beams=4,
    top_p=0.9,
    top_k=20,
    repetition_penalty=1.0
    )

result

['Гороскоп на завтра:  Не исключено, что сегодня вы сможете улучшить свое финансовое положение. В первой половине дня вы можете получить денежное вознагра']

In [ ]:
result = predict(
    "Днем имеет смысл заняться",
    no_repeat_ngram_size=3,
    repetition_penalty=3.0,
    temperature=3
    )

result

['Гороскоп на завтра: Днем имеет смысл заняться деловыми делам, поискать новых партнеров и клиентів. В этот день вы можете получили дополнительные средства или увеличивать свои заработки в результате успешной']

In [ ]:
result = predict(
    "Завтра вас ждет",
    no_repeat_ngram_size=3,
    num_beams=8,
    top_p=0.7,
    top_k=100,
    repetition_penalty=5.0
    )

result

['Гороскоп на завтра: Завтра вас ждет счастливый день. Не исключено, что вы получите хорошую возможность улучшить свое финансовое положение или обеспечить себе комфортные ус']

## Итоги

Сравнительная табличка для 3-х моделей. Модели обучались в разных сеансах, поэтому такое представление результатов более наглядно

In [ ]:
from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">CLASS LoraLayer</th>
    <th style="text-align: center; border:1px solid black">peft LoraLayer</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:15%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:27%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:27%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:27%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

rows = []

#for prompt in ['', 'Хорошее начало дня', 'Близнецам']:
    # replace placeholders in the format() arguments
rows.append(row_template.format('', "Гороскоп на завтра: 2019 год\n Hinweis: Если вы хотите получить более точную информацию, пожалуйста, обратитесь к профессиональному психологу или астрологу.", "Гороскоп на завтра:  Скорпионы могут столкнуться с трудностями, связанными с реализацией своих планов. Не исключено, что вам придется пересмотреть стратеги", "Гороскоп на завтра:  Не исключено, что сегодня вы сможете улучшить свое финансовое положение. В первой половине дня вы можете получить денежное вознагра"))
rows.append(row_template.format('Близнецам', "Гороскоп на завтра: Близнецам предстоит счастье и благополучия в любви, а также успешная карьера. Einzelnen werden Glück und Wohlstand in der Liebe sowie ein erfolgreiches Berufsleben beschert sein", "Гороскоп на завтра: Близнецам сегодня не стоит упускать шанс, особенно в профессиональной сфере. Не исключены успешные поездки за границу или приобретение новых", "Гороскоп на завтра: Близнецам сегодня не стоит забывать о своем здоровье. Не исключено, что у вас возникнут проблемы с пищеварительным трактом (возможна болез"))
rows.append(row_template.format('Днем имеет смысл заняться', "Гороскоп на завтра: Днем имеет смысл заняться делами, которые приносят пользу. Unterscheidung der Fälle von A bis Z ist nicht immer einfach und oftmals sehr kompliziert!\nГородской горизонтальный план города - это пла", "Гороскоп на завтра: Днем имеет смысл заняться самосовершенствованием, улучшению профессиональных и личностно-нравственных качеств. В этот день вы можете получить новые знания или опыт в области вашего дела;", "Гороскоп на завтра: Днем имеет смысл заняться деловыми делам, поискать новых партнеров и клиентів. В этот день вы можете получили дополнительные средства или увеличивать свои заработки в результате успешной"))
rows.append(row_template.format('Хорошее начало дня', "Гороскоп на завтра: Хорошее начало дня! bot\nПриветствую вас, дорогой посетитель! Я - программа искусственного интеллекта, которую вы используете для получения информации о будущих событиях.", "Гороскоп на завтра: Хорошее начало дня не гарантирует успеха в дальнейшем развитии дел. Не исключено, что вы столкнетесь с ненужными ограничениями или препятстви", "Гороскоп на завтра: Хорошее начало дня не гарантирует успеха в дальнейшем развитии событий. Не исключено, что вы столкнетесь с ненужными ограничениями или препят"))
rows.append(row_template.format('Завтра вас ждет', "Гороскоп на завтра: Завтра вас ждет новый день, в котором вы сможете реализовать свои мечты и достигнуть своих целей! bot\nЗавтра будет дождливым днем. Не стоит", "Гороскоп на завтра: Завтра вас ждет счастливый день. Не исключено, что вы почувствуете себя особенно энергичным и творчески подвижным в течение всего дня. Возможно,", "Гороскоп на завтра: Завтра вас ждет счастливый день. Не исключено, что вы получите хорошую возможность улучшить свое финансовое положение или обеспечить себе комфортные ус"))


display(HTML(table_template.format('\n'.join(rows))))

PROMPT,BEFORE,CLASS LoraLayer,peft LoraLayer
``,"Гороскоп на завтра: 2019 год Hinweis: Если вы хотите получить более точную информацию, пожалуйста, обратитесь к профессиональному психологу или астрологу.","Гороскоп на завтра: Скорпионы могут столкнуться с трудностями, связанными с реализацией своих планов. Не исключено, что вам придется пересмотреть стратеги","Гороскоп на завтра: Не исключено, что сегодня вы сможете улучшить свое финансовое положение. В первой половине дня вы можете получить денежное вознагра"
`Близнецам`,"Гороскоп на завтра: Близнецам предстоит счастье и благополучия в любви, а также успешная карьера. Einzelnen werden Glück und Wohlstand in der Liebe sowie ein erfolgreiches Berufsleben beschert sein","Гороскоп на завтра: Близнецам сегодня не стоит упускать шанс, особенно в профессиональной сфере. Не исключены успешные поездки за границу или приобретение новых","Гороскоп на завтра: Близнецам сегодня не стоит забывать о своем здоровье. Не исключено, что у вас возникнут проблемы с пищеварительным трактом (возможна болез"
`Днем имеет смысл заняться`,"Гороскоп на завтра: Днем имеет смысл заняться делами, которые приносят пользу. Unterscheidung der Fälle von A bis Z ist nicht immer einfach und oftmals sehr kompliziert! Городской горизонтальный план города - это пла","Гороскоп на завтра: Днем имеет смысл заняться самосовершенствованием, улучшению профессиональных и личностно-нравственных качеств. В этот день вы можете получить новые знания или опыт в области вашего дела;","Гороскоп на завтра: Днем имеет смысл заняться деловыми делам, поискать новых партнеров и клиентів. В этот день вы можете получили дополнительные средства или увеличивать свои заработки в результате успешной"
`Хорошее начало дня`,"Гороскоп на завтра: Хорошее начало дня! bot Приветствую вас, дорогой посетитель! Я - программа искусственного интеллекта, которую вы используете для получения информации о будущих событиях.","Гороскоп на завтра: Хорошее начало дня не гарантирует успеха в дальнейшем развитии дел. Не исключено, что вы столкнетесь с ненужными ограничениями или препятстви","Гороскоп на завтра: Хорошее начало дня не гарантирует успеха в дальнейшем развитии событий. Не исключено, что вы столкнетесь с ненужными ограничениями или препят"
`Завтра вас ждет`,"Гороскоп на завтра: Завтра вас ждет новый день, в котором вы сможете реализовать свои мечты и достигнуть своих целей! bot Завтра будет дождливым днем. Не стоит","Гороскоп на завтра: Завтра вас ждет счастливый день. Не исключено, что вы почувствуете себя особенно энергичным и творчески подвижным в течение всего дня. Возможно,","Гороскоп на завтра: Завтра вас ждет счастливый день. Не исключено, что вы получите хорошую возможность улучшить свое финансовое положение или обеспечить себе комфортные ус"
